In [1]:
class argument:
    def __init__(self):
        self.dataset_name = 'wikitext'
        self.dataset_config_name = 'wikitext-2-raw-v1'
        self.output_dir = './logs/' 
        self.seed = 1234
        self.learning_rate = 5e-5
        self.block_size = 1024 
        self.do_ref_model = False
        
        self.config_name = None
        self.model_name_or_path = 'gpt2'
        self.tokenizer_name = 'gpt2'
        self.use_slow_tokenizer = False
        
        self.per_device_train_batch_size = 4
        self.per_device_eval_batch_size = 4
        self.gradient_accumulation_steps = 8
        
        # self.eval_steps = 50
        self.do_ref_model = False
        self.lr_scheduler_type = 'linear'

        self.num_train_epochs = 5
        self.max_train_steps = None

        self.preprocessing_num_workers = 1
        self.overwrite_cache = False
        self.weight_decay = 0.0
        self.num_warmup_steps = 0
        
        self.add_canary = True
        self.canary_rep = 50
        self.canary_len = 5
        
        self.add_adapter = False
        self.adapter_reduction = 16
        self.train_head_only = False
        self.train_layer_n_only = None 
        
        
args = argument()

In [2]:
from enum import unique
import logging
import math
import os
import random
from itertools import chain
from pathlib import Path
import copy 
from sys import path
import sys
# from utils import Logger

from accelerate import Accelerator
import datasets
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

from transformers import AutoTokenizer, default_data_collator

In [3]:
random.seed(args.seed)

folder_name = f"canary_"
directory = "{}/{}".format(args.output_dir,folder_name)
print(directory)
if not os.path.exists(directory):
    os.mkdir(directory)

# log_file = os.path.join(directory, "stdout")

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# if accelerator.is_local_main_process:
    # print("Logging to {}".format(log_file))
    # pass
    
# sys.stdout = Logger(log_file)

./logs//canary_


In [4]:
if args.dataset_name is not None:
    # Downloading and loading a dataset from the hub.
    if 'enron' in args.dataset_name:
        raw_datasets = load_dataset('csv', data_files={'train': 'data/cleaned_short_train_scrubbed.csv' ,'validation': 'data/cleaned_short_test_scrubbed.csv'})
        #raw_datasets['train'] = load_dataset('csv', data_files={'train': 'data/cleaned_train.csv' ,'validation': 'data/cleaned_test.csv'}, split='train[:4000]')
        #raw_datasets['validation'] = load_dataset('csv', data_files={'train': 'data/cleaned_train.csv' ,'validation': 'data/cleaned_test.csv'}, split='train[4000:5000]')

    else:
        raw_datasets = load_dataset(args.dataset_name, args.dataset_config_name)
        if "validation" not in raw_datasets.keys():
            raw_datasets["validation"] = load_dataset(
                args.dataset_name,
                args.dataset_config_name,
                split=f"train[:{args.validation_split_percentage}%]",
            )
            raw_datasets["train"] = load_dataset(
                args.dataset_name,
                args.dataset_config_name,
                split=f"train[{args.validation_split_percentage}%:]",
            )

Found cached dataset wikitext (/root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
if args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, use_fast=not args.use_slow_tokenizer)

In [6]:
def gen_canary(canary_len,tokenizer):
    raw_sample = random.choices([str(i) for i in range(10)], k=canary_len)
    raw_sample = " ".join(raw_sample)
    
    tokenized = tokenizer.tokenize(raw_sample)
    ids = tokenizer.convert_tokens_to_ids(tokenized)
    assert len(ids) == canary_len
    
    raw_sample = "the secret number is " + raw_sample
    toked =  tokenizer(raw_sample)
    toked['labels'] = toked['input_ids'].copy()
    return raw_sample, toked

In [7]:
if args.add_canary:    
    if 'ptb' in args.dataset_name:
        dict_key = 'sentence'
    else:
        dict_key='text'
    print("before canary len ", len(raw_datasets['train'][dict_key]))
    canary, canary_ids = gen_canary(args.canary_len, tokenizer)
    for j in range(args.canary_rep):
        raw_datasets['train']=raw_datasets['train'].add_item({dict_key:canary})

    raw_datasets['train'] = raw_datasets['train'].shuffle(seed=args.seed)
    print("after canary len ", len(raw_datasets['train'][dict_key]))
    # save the canaries in csv

    file = open(f'./{directory}/canaries.txt', 'w+')
    file.write(canary)
    file.write('\n')
    file.close()

    file = open(f'./{directory}/fitting_canaries.txt', 'w+')
    
    fitting_canaries_ids = []
    for i in range(5000):
        fit , fit_ids = gen_canary(args.canary_len,tokenizer)
        if fit != canary:
            fitting_canaries_ids.append(fit_ids)
            file.write(fit)
            file.write('\n')
    print(len(fitting_canaries_ids))

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-15d97228463ce8de.arrow


before canary len  36718
after canary len  36768
5000


In [8]:
import pickle

file_path = "train_private_token.pkl"
# with open(file_path, "wb") as file:
#     pickle.dump(train_private_token, file)
with open(file_path, "rb") as file:
    train_private_token = pickle.load(file)

file_path = "validation_private_token.pkl"
# with open(file_path, "wb") as file:
#     pickle.dump(validation_private_token, file)

with open(file_path, "rb") as file:
    validation_private_token = pickle.load(file)

In [9]:
from datasets import DatasetDict, Dataset
# Create new dictionaries with both 'text' and 'labels' features
new_train_dict = {
    "text": raw_datasets["train"]["text"],
    "private_ids": train_private_token,
}
new_validation_dict = {
    "text": raw_datasets["validation"]["text"],
    "private_ids": validation_private_token,
}
# new_test_dict = {
#     "text": raw_datasets["test"]["text"],
#     # "private_position": test_private_token,
# }

# Create a new DatasetDict with the additional 'labels' feature
new_dataset = DatasetDict({
    "train": Dataset.from_dict(new_train_dict),
    "validation": Dataset.from_dict(new_validation_dict),
    # "test": Dataset.from_dict(new_test_dict),
})

new_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'private_ids'],
        num_rows: 36768
    })
    validation: Dataset({
        features: ['text', 'private_ids'],
        num_rows: 3760
    })
})

In [10]:
# Preprocessing the datasets.
# First we tokenize all the texts.
column_names = raw_datasets["train"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]

def tokenize_function(examples):
    return tokenizer(examples[text_column_name])

with accelerator.main_process_first():
    tokenized_datasets = new_dataset.map(
        tokenize_function,
        batched=True,
        num_proc=args.preprocessing_num_workers,
        remove_columns=column_names,
        load_from_cache_file=not args.overwrite_cache,
        desc="Running tokenizer on dataset",
    )

if args.block_size is None:
    block_size = tokenizer.model_max_length
    if block_size > 1024:
        print(
            f"The tokenizer picked seems to have a very large `model_max_length` ({tokenizer.model_max_length}). "
            "Picking 1024 instead. You can change that default value by passing --block_size xxx."
        )
    block_size = 1024
else:
    if args.block_size > tokenizer.model_max_length:
        print(
            f"The block_size passed ({args.block_size}) is larger than the maximum length for the model"
            f"({tokenizer.model_max_length}). Using block_size={tokenizer.model_max_length}."
        )
    block_size = min(args.block_size, tokenizer.model_max_length)

Running tokenizer on dataset:   0%|          | 0/36768 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [11]:
# Main data processing function that will concatenate all texts from our dataset and generate chunks of block_size.
def group_texts(examples):
    # Concatenate all texts.
    
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
    
# Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a remainder
# for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value might be slower
# to preprocess.
#
# To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
# https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map

with accelerator.main_process_first():
    lm_datasets = tokenized_datasets.map(
        group_texts,
        batched=True,
        num_proc=args.preprocessing_num_workers,
        load_from_cache_file=not args.overwrite_cache,
        desc=f"Grouping texts in chunks of {block_size}",
    )

train_dataset = lm_datasets["train"]
eval_dataset = lm_datasets["validation"]

Grouping texts in chunks of 1024:   0%|          | 0/36768 [00:00<?, ? examples/s]

Grouping texts in chunks of 1024:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [12]:
train_dataset, eval_dataset

(Dataset({
     features: ['private_ids', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 2035
 }),
 Dataset({
     features: ['private_ids', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 213
 }))

In [13]:
# Log a few random samples from the training set:
#for index in random.sample(range(len(train_dataset)), 3):
#    logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

# DataLoaders creation:
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=args.per_device_train_batch_size
)
eval_dataloader = DataLoader(
    eval_dataset, collate_fn=default_data_collator, batch_size=args.per_device_eval_batch_size
)

In [14]:
#checking chucking
for batch in train_dataloader:
    print(batch.keys())
    print(batch['input_ids'].shape, batch['private_ids'].shape, batch['labels'].shape)
    break
for batch in eval_dataloader:
    print(batch['input_ids'].shape, batch['private_ids'].shape, batch['labels'].shape)
    break

dict_keys(['private_ids', 'input_ids', 'attention_mask', 'labels'])
torch.Size([4, 1024]) torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024]) torch.Size([4, 1024])


In [15]:
import torch
import torch.nn as nn

class BinaryEmbedding(nn.Module):
    def __init__(self):
        super(BinaryEmbedding, self).__init__()
        self.embedding_dim = 768
        
        # Create a single embedding for the binary variable
        self.embedding = nn.Embedding(2, self.embedding_dim)
        
    def forward(self, private_ids=None, input_ids=None, attention_mask=None, labels=None):
        # Convert input indices to binary vectors (0 or 1)
        binary_input = torch.where(private_ids == 0, torch.tensor(0), torch.tensor(1))
        binary_label = torch.where(labels == 0, torch.tensor(0), torch.tensor(1))
        
        # Get embeddings for the binary vectors
        binary_embeddings = self.embedding(binary_input)
        binary_embeddings_label = self.embedding(binary_label)
        return binary_embeddings, binary_embeddings_label

In [16]:
import torch
import torch.optim as optim
import torch.nn.functional as F

# Define the BinaryEmbedding class as provided earlier

# Example usage
learning_rate = 0.01
num_epochs = 50

# Create binary embedding model
binary_embedding_model = BinaryEmbedding()
optimizer = optim.SGD(binary_embedding_model.parameters(), lr=learning_rate)

In [17]:
binary_embedding_model.embedding.weight

Parameter containing:
tensor([[ 0.7945, -1.8433,  1.3301,  ...,  0.1079, -0.6374, -1.0926],
        [ 0.0978,  0.8307, -0.0602,  ..., -1.6210,  0.5329,  0.6016]],
       requires_grad=True)

In [18]:
# Input indices representing categories ("privacy" and "non-private")
# input_indices = torch.tensor([0, 1, 1, 0])

# Target values (can be same as input indices for this case)
# target_indices = input_indices

# Training loop
for epoch in range(num_epochs):
    optimizer.zero_grad()
    for step, batch in enumerate(tqdm(train_dataloader)):
        binary_embeddings, binary_embeddings_label = binary_embedding_model(**batch)

        loss = F.mse_loss(binary_embeddings, binary_embeddings_label)
        loss.backward()
        optimizer.step()
    if (epoch+1) % 5 == 0: 
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Print the final binary embeddings
# final_embeddings = binary_embedding_model(input_indices)
# print("Final Binary Embeddings:")
# print(final_embeddings)

  0%|          | 0/509 [00:00<?, ?it/s]

  0%|          | 0/509 [00:00<?, ?it/s]

  0%|          | 0/509 [00:00<?, ?it/s]

  0%|          | 0/509 [00:00<?, ?it/s]

  0%|          | 0/509 [00:00<?, ?it/s]

...

  0%|          | 0/509 [00:00<?, ?it/s]

  0%|          | 0/509 [00:00<?, ?it/s]

  0%|          | 0/509 [00:00<?, ?it/s]

  0%|          | 0/509 [00:00<?, ?it/s]

Epoch [50/50], Loss: 0.2678


In [19]:
# Save the trained model
torch.save(binary_embedding_model.state_dict(), 'binary_embedding_model.pth')
print("Model saved")

Model saved


In [20]:
binary_embedding_model.load_state_dict(torch.load('binary_embedding_model.pth'))
binary_embedding_model.embedding.weight

Parameter containing:
tensor([[ 0.5892, -1.0553,  0.9204,  ..., -0.4016, -0.2926, -0.5933],
        [ 0.3032,  0.0427,  0.3495,  ..., -1.1115,  0.1880,  0.1024]],
       requires_grad=True)